# TNT Cates: Large-Amplitude Oscillatory Shear (LAOS)
> **Handbook:** This notebook demonstrates the TNT Cates model. For complete mathematical derivations and theoretical background, see [TNT Cates Documentation](../../docs/source/models/tnt/tnt_cates.rst).


**Estimated Time:** 4-6 minutes

## Protocol: LAOS in Living Polymers

**LAOS** in living polymers reveals **strong nonlinearity** from stress-induced chain scission and potential **shear banding** instabilities.

### Physical Context for Living Polymers

At large strain amplitudes, wormlike micelles exhibit:

1. **Strain-dependent scission**: High shear stress → increased chain breaking rate → shorter average length
2. **Flow-induced alignment**: Chains orient along flow direction, reducing entanglements
3. **Shear banding**: Coexistence of low-shear (structured) and high-shear (flow-aligned) bands

The Cates model captures:
- **Higher harmonics**: $G'_3, G''_3, G'_5, ...$ from nonlinear conformation tensor evolution
- **Lissajous distortion**: Non-elliptical stress-strain curves (strain-softening)
- **Critical strain amplitude**: Onset of shear banding (material instability)

Contrast with:
- **TNT Single-Mode**: Weak geometric nonlinearity (strain-stiffening only)
- **Loop-Bridge**: Shear thickening from bridge formation under strain

> **Cates LAOS Physics**  
> For nonlinear harmonics and shear banding analysis, see:  
> [../../docs/source/models/tnt/tnt_cates.rst](../../docs/source/models/tnt/tnt_cates.rst) — Section on "LAOS and Shear Banding"

---

## Learning Objectives

1. Understand nonlinear rheology and shear banding in living polymers
2. Fit TNT Cates model to LAOS stress waveforms
3. Extract higher harmonics $G'_3, G''_3$ from Fourier analysis
4. Analyze Lissajous curves for strain-softening signatures
5. Predict critical strain for shear banding onset
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- TNT Cates fundamentals (Notebook 07, 11)
- Understanding of LAOS (TNT Single-Mode NB06)
- Familiarity with shear banding phenomena

## Runtime Estimate

- NLSQ fitting: ~8-15 seconds (ODE integration)
- Bayesian inference (demo): ~90-180 seconds
- Total: ~8-12 minutes

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_pnas_laos,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_cates_param_names,
    compute_cates_tau_d,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Cates Model for LAOS

The Cates model predicts nonlinear oscillatory response in living polymers:

**LAOS protocol:**
- Applied strain: $\gamma(t) = \gamma_0 \sin(\omega t)$
- Measure stress: $\sigma(t)$ (nonlinear waveform)

**Shear banding threshold:**
When $\sigma$ exceeds the maximum stress in the flow curve:
$$\sigma_{\text{max}} = \frac{2G_0\tau_d}{3\sqrt{3}}$$

the material may undergo transient shear banding.

**Nonlinear signatures:**
- Stress plateau at high strain amplitudes
- Odd harmonics in FFT (3rd, 5th, 7th, ...)
- Distorted Lissajous curves (stress vs strain)

**Physical interpretation:**
- Higher harmonics indicate departure from linear viscoelasticity
- Cage breaking and reformation dynamics during oscillation

## Load LAOS Data

In [ ]:
omega_laos = 1.0
gamma_0 = 0.2  # Approximate strain amplitude
time_data, strain, stress = load_pnas_laos(omega=omega_laos, strain_amplitude_index=5)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2f} to {time_data.max():.2f} s")
print(f"Applied frequency: {omega_laos:.3f} rad/s")
print(f"Applied strain amplitude: {gamma_0:.3f}")
print(f"Stress range: {stress.min():.2f} to {stress.max():.2f} Pa")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, stress, '-', linewidth=1, label='Stress')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Stress (Pa)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('LAOS Stress Time Series', fontsize=14)

ax2.plot(strain, stress, '-', linewidth=1)
ax2.set_xlabel('Strain γ', fontsize=12)
ax2.set_ylabel('Stress σ (Pa)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_title('Lissajous Curve', fontsize=14)

plt.tight_layout()
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

start_time = time.time()
model.fit(time_data, stress, test_mode='laos', omega=omega_laos, gamma_0=gamma_0, method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
stress_pred_fit = model.predict(time_data, test_mode='laos', omega=omega_laos, gamma_0=gamma_0)
quality = compute_fit_quality(stress, stress_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
time_pred = jnp.linspace(time_data.min(), time_data.max(), 500)
stress_pred = model.predict(time_pred, test_mode='laos', omega=omega_laos, gamma_0=gamma_0)

strain_pred = gamma_0 * jnp.sin(omega_laos * time_pred)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, stress, 'o', label='Data', markersize=3, alpha=0.5)
ax1.plot(time_pred, stress_pred, '-', linewidth=2, label='TNTCates fit')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Stress (Pa)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('LAOS Time Series Fit', fontsize=14)

ax2.plot(strain, stress, 'o', label='Data', markersize=3, alpha=0.5)
ax2.plot(strain_pred, stress_pred, '-', linewidth=2, label='TNTCates fit')
ax2.set_xlabel('Strain γ', fontsize=12)
ax2.set_ylabel('Stress σ (Pa)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_title('Lissajous Curve Fit', fontsize=14)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Physical Analysis: Shear Banding Threshold

In [ ]:
tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

sigma_max_theory = (2 * nlsq_params['G_0'] * tau_d) / (3 * np.sqrt(3))
sigma_max_observed = np.max(np.abs(stress_pred))

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")
print(f"\nShear Banding Analysis:")
print(f"  Theoretical max stress (σ_max): {sigma_max_theory:.2f} Pa")
print(f"  Observed max stress: {sigma_max_observed:.2f} Pa")
print(f"  Stress ratio (σ_obs/σ_max): {sigma_max_observed/sigma_max_theory:.3f}")

if sigma_max_observed > sigma_max_theory:
    print(f"  ⚠ Stress exceeds σ_max: Potential transient shear banding")
else:
    print(f"  ✓ Stress below σ_max: No shear banding expected")

## FFT Harmonics Analysis

In [ ]:
dt = time_pred[1] - time_pred[0]
stress_fft = fft(np.array(stress_pred))
freqs = fftfreq(len(time_pred), float(dt))

positive_mask = freqs > 0
freqs_pos = freqs[positive_mask]
amplitudes = 2.0 * np.abs(stress_fft[positive_mask]) / len(time_pred)

fundamental_freq = omega_laos / (2 * np.pi)
harmonic_indices = []
harmonic_amplitudes = []
for n in [1, 3, 5, 7]:
    target_freq = n * fundamental_freq
    idx = np.argmin(np.abs(freqs_pos - target_freq))
    harmonic_indices.append(n)
    harmonic_amplitudes.append(amplitudes[idx])

print(f"\nFFT Harmonic Analysis:")
print(f"  Fundamental frequency: {fundamental_freq:.4f} Hz")
for n, amp in zip(harmonic_indices, harmonic_amplitudes):
    print(f"  Harmonic {n}: {amp:.2f} Pa ({amp/harmonic_amplitudes[0]*100:.1f}% of fundamental)")

fig, ax = plt.subplots(figsize=(10, 5))
ax.stem([n * fundamental_freq for n in harmonic_indices], harmonic_amplitudes, basefmt=' ')
ax.set_xlabel('Frequency (Hz)', fontsize=12)
ax.set_ylabel('Amplitude (Pa)', fontsize=12)
ax.set_xlim(0, 8 * fundamental_freq)
ax.grid(True, alpha=0.3)
ax.set_title('LAOS Harmonics', fontsize=14)
plt.close("all")
plt.close('all')

## Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        time_data, stress,
        test_mode='laos',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
if FAST_MODE:
    print("FAST_MODE: Skipping posterior predictive (200 ODE predictions would take >300s)")
    # Single NLSQ prediction for plot
    stress_pred_final = model.predict(time_pred, test_mode='laos', omega=omega_laos, gamma_0=gamma_0)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.plot(time_data, stress, 'o', label='Data', markersize=3, alpha=0.5)
    ax1.plot(time_pred, stress_pred_final, '-', linewidth=2, label='NLSQ fit')
    ax1.set_xlabel('Time (s)', fontsize=12)
    ax1.set_ylabel('Stress (Pa)', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_title('NLSQ Fit (CI Mode)', fontsize=14)
    
    ax2.plot(strain, stress, 'o', label='Data', markersize=3, alpha=0.5)
    ax2.plot(strain_pred, stress_pred_final, '-', linewidth=2, label='NLSQ fit')
    ax2.set_xlabel('Strain γ', fontsize=12)
    ax2.set_ylabel('Stress σ (Pa)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_title('Lissajous (CI Mode)', fontsize=14)
    
    plt.tight_layout()
    plt.close("all")
    plt.close('all')
else:
    n_posterior_samples = 200
    sample_indices = np.random.choice(NUM_SAMPLES * NUM_CHAINS, n_posterior_samples, replace=False)
    
    predictions = []
    for idx in sample_indices:
        # Set model parameters from posterior sample
        for j, name in enumerate(param_names):
            model.parameters.set_value(name, float(posterior[name].flatten()[idx]))
        # Predict with updated parameters
        pred_i = model.predict(time_pred, test_mode='laos', omega=omega_laos, gamma_0=gamma_0)
        predictions.append(np.array(pred_i))
    
    predictions = np.array(predictions)
    pred_mean = np.mean(predictions, axis=0)
    pred_lower = np.percentile(predictions, 2.5, axis=0)
    pred_upper = np.percentile(predictions, 97.5, axis=0)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.plot(time_data, stress, 'o', label='Data', markersize=3, alpha=0.5, zorder=3)
    ax1.plot(time_pred, pred_mean, '-', linewidth=2, label='Posterior mean', zorder=2)
    ax1.fill_between(time_pred, pred_lower, pred_upper, alpha=0.3, label='95% CI', zorder=1)
    ax1.set_xlabel('Time (s)', fontsize=12)
    ax1.set_ylabel('Stress (Pa)', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Posterior Predictive (Time Series)', fontsize=14)
    
    ax2.plot(strain, stress, 'o', label='Data', markersize=3, alpha=0.5, zorder=3)
    ax2.plot(strain_pred, pred_mean, '-', linewidth=2, label='Posterior mean', zorder=2)
    ax2.set_xlabel('Strain γ', fontsize=12)
    ax2.set_ylabel('Stress σ (Pa)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_title('Posterior Predictive (Lissajous)', fontsize=14)
    
    plt.tight_layout()
    plt.close("all")
    plt.close('all')

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']
sigma_max_posterior = (2 * posterior['G_0'] * tau_d_posterior) / (3 * np.sqrt(3))

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")
print(f"  σ_max (shear banding threshold): {np.mean(sigma_max_posterior):.2f} ± {np.std(sigma_max_posterior):.2f} Pa")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(sigma_max_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(sigma_max_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(sigma_max_observed, color='g', linestyle=':', label='Observed max')
ax2.set_xlabel(r'$\sigma_{max}$ (Pa)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Shear Banding Threshold Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "laos", param_names)

## Key Takeaways

1. **Nonlinear LAOS response** shows stress plateau and odd harmonics for living polymers
2. **Shear banding threshold** $\sigma_{\text{max}} = 2G_0\tau_d/(3\sqrt{3})$ predicts when transient banding occurs
3. **Odd harmonics** (3rd, 5th, 7th) indicate departure from linear viscoelasticity
4. **Lissajous distortion** reveals nonlinear cage dynamics under large-amplitude oscillation
5. **Bayesian inference** quantifies uncertainty in parameters from complex LAOS waveforms

**Cross-protocol validation:** Compare extracted $\tau_d$ with flow curve, startup, relaxation, creep, and SAOS to ensure global consistency of Cates model parameters.

## Next Steps

- **Notebook 07**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Cates Series Complete

You've completed all 6 protocols for the TNT Cates model (living polymers / wormlike micelles). This sub-family demonstrates **two-timescale physics** from reptation + reversible scission, with the signature **geometric mean relaxation time** $\tau_d = \sqrt{\tau_{rep} \tau_{break}}$.

### TNT Cates Documentation

- **[TNT Cates Handbook](../../docs/source/models/tnt/tnt_cates.rst)**: Detailed physics, equations, Cole-Cole analysis, shear banding
- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Comparison to other TNT models
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Extracting $\tau_{rep}$ and $\tau_{break}$ from $\tau_d$

### Next: Explore Other TNT Models

**Loop-Bridge Model (NB13-18)**: Shear thickening/thinning networks
- **Key physics**: Intra-chain loops vs. inter-chain bridges, only bridges carry stress
- **Signatures**: Non-monotonic viscosity, shear thickening then thinning at high rates
- **Handbook**: [../../docs/source/models/tnt/tnt_loop_bridge.rst](../../docs/source/models/tnt/tnt_loop_bridge.rst)

**Multi-Species Model (NB19-24)**: Multiple bond types
- **Key physics**: Independent species with different lifetimes, total stress = sum
- **Signatures**: Staged shear thinning, multi-modal relaxation spectrum
- **Handbook**: [../../docs/source/models/tnt/tnt_multi_species.rst](../../docs/source/models/tnt/tnt_multi_species.rst)

**Sticky Rouse Model (NB25-30)**: Hierarchical sticker dynamics
- **Key physics**: Multi-mode Rouse + sticky dynamics, $\tau_{eff,k} = \tau_{R,k} + \tau_s$
- **Signatures**: Power-law $G'(\omega) \sim \omega^{1/2}$ Rouse regime, sticky plateau
- **Handbook**: [../../docs/source/models/tnt/tnt_sticky_rouse.rst](../../docs/source/models/tnt/tnt_sticky_rouse.rst)

### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
